In [1]:
!pip install -U "transformers>=4.40" "datasets>=2.19" "accelerate>=0.30" scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Unins

In [2]:
# upload
from google.colab import files
files.upload()  # select train_goemotions_distilbert_colab_v2.py


Saving train_goemotions_distilbert_colab_v2.py to train_goemotions_distilbert_colab_v2.py


{'train_goemotions_distilbert_colab_v2.py': b'#!/usr/bin/env python3\n"""\nColab-safe GoEmotions baseline (DistilBERT) \xe2\x80\x94 v2\n- Handles both list-based and wide one-hot schemas.\n- Auto-creates a validation split if the dataset lacks one.\n- Minimal TrainingArguments to sidestep version quirks.\n"""\n\nimport argparse\nimport json\nimport os\nimport time\nfrom typing import List\n\nimport numpy as np\nimport torch\nfrom datasets import load_dataset, DatasetDict, Sequence\nfrom transformers import (\n    AutoTokenizer,\n    AutoModelForSequenceClassification,\n    DataCollatorWithPadding,\n    Trainer,\n    TrainingArguments,\n)\nfrom sklearn.metrics import f1_score, classification_report\n\n\nRAW_EMOTIONS = [\n    "admiration","amusement","anger","annoyance","approval","caring","confusion",\n    "curiosity","desire","disappointment","disapproval","disgust","embarrassment",\n    "excitement","fear","gratitude","grief","joy","love","nervousness","optimism",\n    "pride","realiz

In [3]:
# train (auto-splits 10% of train into validation if missing)
!python train_goemotions_distilbert_colab_v2.py --dataset_name go_emotions --dataset_config raw --val_frac 0.1
# or the simplified setup:
# !python train_goemotions_distilbert_colab_v2.py --dataset_name go_emotions --dataset_config simplified --val_frac 0.1


2025-09-26 15:24:40.675912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758900280.695108    2365 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758900280.700885    2365 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758900280.717188    2365 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758900280.717211    2365 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758900280.717216    2365 computation_placer.cc:177] computation placer alr

In [4]:
from datasets import load_dataset
ds = load_dataset("go_emotions","raw")
print(ds)
print(ds["train"].column_names)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
        num_rows: 211225
    })
})
['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [5]:
import json, pprint, os
base="outputs_distilbert_goemotions"
pprint.pp(json.load(open(os.path.join(base,"metrics.json"))))


{'train': {'train_runtime': 2388.7815,
           'train_samples_per_second': 238.743,
           'train_steps_per_second': 7.461,
           'total_flos': 5938392791853312.0,
           'train_loss': 0.10944720140739783,
           'epoch': 3.0},
 'val': {'eval_loss': 0.11007603257894516,
         'eval_f1_micro': 0.3757108380260201,
         'eval_f1_macro': 0.2916099919553024,
         'eval_f1_weighted': 0.3440476728150121,
         'eval_runtime': 25.0593,
         'eval_samples_per_second': 842.919,
         'eval_steps_per_second': 26.377,
         'epoch': 3.0},
 'test': {'eval_loss': 0.11007603257894516,
          'eval_f1_micro': 0.3757108380260201,
          'eval_f1_macro': 0.2916099919553024,
          'eval_f1_weighted': 0.3440476728150121,
          'eval_runtime': 25.2427,
          'eval_samples_per_second': 836.796,
          'eval_steps_per_second': 26.186,
          'epoch': 3.0},
 'label_names': ['admiration',
                 'amusement',
                 'anger',

In [15]:
!pip install reportlab matplotlib pandas
from google.colab import files; files.upload()  # select make_report.py


Saving make_report.py to make_report.py


{'make_report.py': b'#!/usr/bin/env python3\nimport os, json, argparse, sys\nfrom datetime import datetime\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom reportlab.lib.pagesizes import A4\nfrom reportlab.lib.units import cm\nfrom reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle\nfrom reportlab.lib.styles import getSampleStyleSheet\nfrom reportlab.lib import colors\n\ndef safe_get(metric_block, keys):\n    for k in keys:\n        if k in metric_block and metric_block[k] is not None:\n            return metric_block[k]\n    return None\n\ndef load_metrics(metrics_path, report_path):\n    METRICS = json.load(open(metrics_path))\n    REPORT  = json.load(open(report_path))\n    debug = {"val_keys": [], "test_keys": []}\n\n    val = METRICS.get("val", {}) or {}\n    test = METRICS.get("test", {}) or {}\n\n    # First try to read from metrics.json\n    val_micro = safe_get(val, ["f1_micro", "eval_f1_micro"])\n    val_macro = safe_get(val, [

In [16]:
# generate the PDF
!python make_report.py --outputs_dir ./outputs_distilbert_goemotions --pdf_path ./goemotions_baseline_report.pdf


# download a single file (no zip)
from google.colab import files
files.download("goemotions_baseline_report.pdf")


/content/make_report.py:117: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  story.append(Paragraph(datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC"), body))
Wrote PDF to: ./goemotions_baseline_report.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>